In [16]:
import random
import math
from environment import Agent, Environment
from planner import RoutePlanner
from simulator import Simulator
# import numpy as np

class LearningAgent(Agent):
    """ An agent that learns to drive in the Smartcab world.
        This is the object you will be modifying. """ 

    def __init__(self, env, learning=False, epsilon=1.0, alpha=0.5):
        super(LearningAgent, self).__init__(env)     # Set the agent in the evironment 
        self.planner = RoutePlanner(self.env, self)  # Create a route planner
        self.valid_actions = self.env.valid_actions  # The set of valid actions

        # Set parameters of the learning agent
        self.learning = learning # Whether the agent is expected to learn
        self.Q = dict()          # Create a Q-table which will be a dictionary of tuples
        self.epsilon = epsilon   # Random exploration factor
        self.alpha = alpha       # Learning factor

        ###########
        ## TO DO ##
        ###########
        # Set any additional class parameters as needed
        self.trial = 0.0



    def reset(self, destination=None, testing=True):
        """ The reset function is called at the beginning of each trial.
            'testing' is set to True if testing trials are being used
            once training trials have completed. """

        # Select the destination as the new location to route to
        self.planner.route_to(destination)
        
        ########### 
        ## TO DO ##
        ###########
        # Update epsilon using a decay function of your choice
        # Update additional class parameters as needed
        # If 'testing' is True, set epsilon and alpha to 0
                # If 'testing' is True, set epsilon and alpha to 0
#         if testing==True:
#             self.epsilon = 0.0 
#             self.alpha = 0.0
#         else:
#             if testing ==False and self.trial<=10.0:
#                 self.alpha = .5
#                 self.trial = 10.0
#                 self.epsilon = self.epsilon 
#                 self.epsilon = 0.8 *.97
#             if testing == False and self.trial ==12.0:
#                 self.trial >=12.0
#                 self.epsilon= self.epsilon
#                 self.epsilon = .5*.96
#                 self.alpha = .5
#                 self.epsilon = 1/self.trial**2.0
#             if  testing ==False and self.trial>=15.0:
#                 self.alpha = .5
#                 self.trial = self.trial
#                 self.epsilon = np.cos(self.alpha*self.trial)
#             if  testing == False and self.trial <= 20.0:
#                 self.trial = self.trial
#                 self.alpha = .5
#                 self.epsilon += np.exp(-1*(self.alpha*self.trial))

        self.alpha = 0.015

        self.trial = self.trial+1

        if testing==True:
            self.epsilon = 0.0 
            self.alpha = 0.0
        elif testing == False:
#             self.epsilon = self.epsilon - .05
#             self.alpha = .5
#             self.trial += 1.0
#             self.epsilon = self.epsilon 
#             self.epsilon = self.epsilon*.25
#             self.epsilon = self.alpha**self.trial
#             self.epsilon = 1.0/self.trial**2.0
#             self.epsilon = np.cos(self.alpha*self.trial)
#             self.epsilon = np.exp(-1*(self.alpha*self.trial))
            self.epsilon = math.exp(-(self.alpha*self.trial))           
        
        return None

    def build_state(self):
        """ The build_state function is called when the agent requests data from the 
            environment. The next waypoint, the intersection inputs, and the deadline 
            are all features available to the agent. """

        # Collect data about the environment
        waypoint = self.planner.next_waypoint() # The next waypoint 
        inputs = self.env.sense(self)           # Visual input - intersection light and traffic
        deadline = self.env.get_deadline(self)  # Remaining deadline
          
       
        state = waypoint, inputs['light'], inputs['oncoming'], inputs['left']
#         if self.learning == True:
#             if state not in self.Q.keys():
#                 self.createQ(state)
#             else:
#                 state.learning ==False

        if self.learning ==True:
            self.createQ(state)

        return state

    def get_maxQ(self, state):
        """ The get_max_Q function is called when the agent is asked to find the
            maximum Q-value of all actions based on the 'state' the smartcab is in. """

        ########### 
        ## TO DO ##
        ###########
        # Calculate the maximum Q-value of all actions for a given state
#         state = self.Q[state][action]

#         max = max(state, key = lambda x: state[x])
#         for action in state:

#             maxQ <= self.Q[state][action]:
#                maxQ = self.Q(state[max] and [action])
        maxQ = max(self.Q[state].values())

        return maxQ 

    def createQ(self, state):
        """ The createQ function is called when a state is generated by the agent. """

        ########### 
        ## TO DO ##
        ###########
        # When learning, check if the 'state' is not in the Q-table
        # If it is not, create a new dictionary for that state
        #   Then, for each action available, set the initial Q-value to 0.0

#         if not self.learning:
#             state = self.valid_actions(state,{'left':0.0,'oncoming':0.25,'forward':0.25,'right':0.0}) #'None','forward'

#         if self.learning == True and state not in self.Q:
#             self.Q[state]={key: 0 for key in self.valid_actions}

        if self.learning==True and state not in self.Q:

            self.Q[state] = {action: 0 for action in self.valid_actions}

        return




    def choose_action(self, state):
        """ The choose_action function is called when the agent is asked to choose
            which action to take, based on the 'state' the smartcab is in. """

        # Set the agent state and default action
        self.state = state
        self.next_waypoint = self.planner.next_waypoint()
        action = None

        if self.learning:

            if self.epsilon > random.random():
                action = random.choice(self.valid_actions)

            else:
                maxQ = self.get_maxQ(state)
                action = random.choice([action for action in self.valid_actions if self.Q[state][action]==maxQ])

        if self.learning==False:

            action = random.choice(self.valid_actions)
        return action


    def learn(self, state, action, reward):
        """ The learn function is called after the agent completes an action and
            receives a reward. This function does not consider future rewards 
            when conducting learning. """

        # When learning, implement the value iteration update rule
        #   Use only the learning rate 'alpha' (do not use the discount factor 'gamma')

        if self.learning == True:
            self.Q[state][action] = (1 - self.alpha) * self.Q[state][action] + self.alpha * reward
#         if self.learning==True:
#             self.Q[state][action] = self.alpha*(reward-self.Q[state][action])
    

        return


    def update(self):
        """ The update function is called when a time step is completed in the 
            environment for a given trial. This function will build the agent
            state, choose an action, receive a reward, and learn if enabled. """

        state = self.build_state()          # Get current state
        self.createQ(state)                 # Create 'state' in Q-table
        action = self.choose_action(state)  # Choose an action
        reward = self.env.act(self, action) # Receive a reward
        self.learn(state, action, reward)   # Q-learn

        return
        

def run():
    """ Driving function for running the simulation. 
        Press ESC to close the simulation, or [SPACE] to pause the simulation. """

    ##############
    # Create the environment
    # Flags:
    #   verbose     - set to True to display additional output from the simulation
    #   num_dummies - discrete number of dummy agents in the environment, default is 100
    #   grid_size   - discrete number of intersections (columns, rows), default is (8, 6)
    env = Environment(verbose=True)
    
    ##############
    # Create the driving agent
    # Flags:
    #   learning   - set to True to force the driving agent to use Q-learning
    #    * epsilon - continuous value for the exploration factor, default is 1
    #    * alpha   - continuous value for the learning rate, default is 0.5
    agent = env.create_agent(LearningAgent, learning=True)#,alpha = .6,epsilon=.75
    
    ##############
    # Follow the driving agent
    # Flags:
    #   enforce_deadline - set to True to enforce a deadline metric
    env.set_primary_agent(agent,enforce_deadline=True)

    ##############
    # Create the simulation
    # Flags:
    #   update_delay - continuous time (in seconds) between actions, default is 2.0 seconds
    #   display      - set to False to disable the GUI if PyGame is enabled
    #   log_metrics  - set to True to log trial and simulation results to /logs
    #   optimized    - set to True to change the default log file name
    sim = Simulator(env,update_delay=.00015,log_metrics=True,display=False,optimized=True)
    
    ##############
    # Run the simulator
    # Flags:
    #   tolerance  - epsilon tolerance before beginning testing, default is 0.05 
    #   n_test     - discrete number of testing trials to perform, default is 0
    sim.run(n_test=20, tolerance =.02)#tolerance=.02


if __name__ == '__main__':
    run()



/-------------------------
| Training trial 1
\-------------------------

Environment.reset(): Trial set up with start = (5, 7), destination = (2, 4), deadline = 30
Simulating trial. . . 
epsilon = 0.9851; alpha = 0.0150

/-------------------
| Step 0 Results
\-------------------

Environment.step(): t = 0
Environment.act() [POST]: location: (5, 2), heading: (0, 1), action: right, reward: 2.30655995489
Environment.act(): Step data: {'inputs': {'light': 'green', 'oncoming': 'left', 'right': 'forward', 'left': None}, 'violation': 0, 'light': 'green', 'state': ('right', 'green', 'left', None), 'deadline': 30, 't': 0, 'action': 'right', 'reward': 2.30655995489312, 'waypoint': 'right'}
Agent previous state: ('right', 'green', 'left', None)
Agent followed the waypoint right. (rewarded 2.31)
97% of time remaining to reach destination.

/-------------------
| Step 1 Results
\-------------------

Environment.step(): t = 1
Environment.act() [POST]: location: (5, 2), heading: (0, 1), action: for

Agent previous state: ('forward', 'green', 'left', None)
Agent idled at a green light with no oncoming traffic. (rewarded -4.45)
57% of time remaining to reach destination.

/-------------------
| Step 13 Results
\-------------------

Environment.step(): t = 13
Environment.act() [POST]: location: (3, 3), heading: (0, -1), action: right, reward: 1.46112438566
Environment.act(): Step data: {'inputs': {'light': 'red', 'oncoming': None, 'right': 'forward', 'left': None}, 'violation': 0, 'light': 'red', 'state': ('forward', 'red', None, None), 'deadline': 17, 't': 13, 'action': 'right', 'reward': 1.461124385655295, 'waypoint': 'forward'}
Agent previous state: ('forward', 'red', None, None)
Agent drove right instead of forward. (rewarded 1.46)
53% of time remaining to reach destination.

/-------------------
| Step 14 Results
\-------------------

Environment.step(): t = 14
Environment.act() [POST]: location: (3, 3), heading: (0, -1), action: None, reward: 2.55070756475
Environment.act(): St

Agent previous state: ('left', 'red', None, None)
Agent properly idled at a red light. (rewarded 2.04)
7% of time remaining to reach destination.

/-------------------
| Step 28 Results
\-------------------

Environment.step(): t = 28
Environment.act() [POST]: location: (3, 3), heading: (-1, 0), action: left, reward: 0.498762442416
Environment.act(): Step data: {'inputs': {'light': 'green', 'oncoming': None, 'right': None, 'left': 'forward'}, 'violation': 0, 'light': 'green', 'state': ('left', 'green', None, 'forward'), 'deadline': 2, 't': 28, 'action': 'left', 'reward': 0.49876244241608414, 'waypoint': 'left'}
Agent previous state: ('left', 'green', None, 'forward')
Agent followed the waypoint left. (rewarded 0.50)
3% of time remaining to reach destination.

/-------------------
| Step 29 Results
\-------------------

Environment.step(): t = 29
Environment.act() [POST]: location: (3, 2), heading: (0, -1), action: right, reward: 0.786353124052
Environment.act(): Step data: {'inputs': {

Agent previous state: ('right', 'red', None, 'right')
Agent properly idled at a red light. (rewarded 1.59)
35% of time remaining to reach destination.

/-------------------
| Step 13 Results
\-------------------

Environment.step(): t = 13
Environment.act() [POST]: location: (5, 5), heading: (0, -1), action: forward, reward: -0.0978244945172
Environment.act(): Step data: {'inputs': {'light': 'green', 'oncoming': 'forward', 'right': 'left', 'left': None}, 'violation': 0, 'light': 'green', 'state': ('right', 'green', 'forward', None), 'deadline': 7, 't': 13, 'action': 'forward', 'reward': -0.0978244945172061, 'waypoint': 'right'}
Agent previous state: ('right', 'green', 'forward', None)
Agent drove forward instead of right. (rewarded -0.10)
30% of time remaining to reach destination.

/-------------------
| Step 14 Results
\-------------------

Environment.step(): t = 14
Environment.act() [POST]: location: (5, 5), heading: (0, -1), action: None, reward: -5.40109278527
Environment.act(): 

Agent previous state: ('left', 'red', None, 'forward')
Agent attempted driving right through traffic and cause a minor accident. (rewarded -19.64)
72% of time remaining to reach destination.

/-------------------
| Step 7 Results
\-------------------

Environment.step(): t = 7
Environment.act() [POST]: location: (7, 7), heading: (0, 1), action: forward, reward: -40.4238102744
Environment.act(): Step data: {'inputs': {'light': 'red', 'oncoming': None, 'right': 'forward', 'left': 'left'}, 'violation': 4, 'light': 'red', 'state': ('left', 'red', None, 'left'), 'deadline': 18, 't': 7, 'action': 'forward', 'reward': -40.42381027442065, 'waypoint': 'left'}
Agent previous state: ('left', 'red', None, 'left')
Agent attempted driving forward through a red light with traffic and cause a major accident. (rewarded -40.42)
68% of time remaining to reach destination.

/-------------------
| Step 8 Results
\-------------------

Environment.step(): t = 8
Environment.act() [POST]: location: (7, 7), hea

...

Agent previous state: ('forward', 'green', None, 'forward')
Agent followed the waypoint forward. (rewarded 2.52)
80% of time remaining to reach destination.

Trial Completed!
Agent reached the destination.

/-------------------------
| Testing trial 16
\-------------------------

Environment.reset(): Trial set up with start = (1, 7), destination = (6, 6), deadline = 20
Simulating trial. . . 
epsilon = 0.0000; alpha = 0.0000

/-------------------
| Step 0 Results
\-------------------

Environment.step(): t = 0
Environment.act() [POST]: location: (1, 7), heading: (1, 0), action: None, reward: 2.34286264942
Environment.act(): Step data: {'inputs': {'light': 'red', 'oncoming': None, 'right': None, 'left': None}, 'violation': 0, 'light': 'red', 'state': ('left', 'red', None, None), 'deadline': 20, 't': 0, 'action': None, 'reward': 2.3428626494180773, 'waypoint': 'left'}
Agent previous state: ('left', 'red', None, None)
Agent properly idled at a red light. (rewarded 2.34)
95% of time remaini

Agent previous state: ('left', 'red', 'left', 'forward')
Agent properly idled at a red light. (rewarded 1.23)
30% of time remaining to reach destination.

/-------------------
| Step 14 Results
\-------------------

Environment.step(): t = 14
Environment.act() [POST]: location: (7, 5), heading: (0, -1), action: None, reward: 2.28859178962
Environment.act(): Step data: {'inputs': {'light': 'red', 'oncoming': 'left', 'right': None, 'left': 'right'}, 'violation': 0, 'light': 'red', 'state': ('left', 'red', 'left', 'right'), 'deadline': 6, 't': 14, 'action': None, 'reward': 2.288591789620517, 'waypoint': 'left'}
Agent previous state: ('left', 'red', 'left', 'right')
Agent properly idled at a red light. (rewarded 2.29)
25% of time remaining to reach destination.

/-------------------
| Step 15 Results
\-------------------

Environment.step(): t = 15
Environment.act() [POST]: location: (7, 5), heading: (0, -1), action: None, reward: 2.08414467828
Environment.act(): Step data: {'inputs': {'li

Agent previous state: ('right', 'green', None, None)
Agent followed the waypoint right. (rewarded 1.75)
96% of time remaining to reach destination.

/-------------------
| Step 1 Results
\-------------------

Environment.step(): t = 1
Environment.act() [POST]: location: (5, 2), heading: (-1, 0), action: right, reward: 1.46968816473
Environment.act(): Step data: {'inputs': {'light': 'red', 'oncoming': 'forward', 'right': None, 'left': 'right'}, 'violation': 0, 'light': 'red', 'state': ('right', 'red', 'forward', 'right'), 'deadline': 24, 't': 1, 'action': 'right', 'reward': 1.4696881647297761, 'waypoint': 'right'}
Agent previous state: ('right', 'red', 'forward', 'right')
Agent followed the waypoint right. (rewarded 1.47)
92% of time remaining to reach destination.

/-------------------
| Step 2 Results
\-------------------

Environment.step(): t = 2
Environment.act() [POST]: location: (5, 7), heading: (0, -1), action: right, reward: 0.633177582925
Environment.act(): Step data: {'inputs

Agent previous state: ('forward', 'green', None, None)
Agent followed the waypoint forward. (rewarded 1.07)
80% of time remaining to reach destination.

/-------------------
| Step 5 Results
\-------------------

Environment.step(): t = 5
Environment.act() [POST]: location: (8, 3), heading: (1, 0), action: None, reward: 2.56436148151
Environment.act(): Step data: {'inputs': {'light': 'red', 'oncoming': None, 'right': 'forward', 'left': 'forward'}, 'violation': 0, 'light': 'red', 'state': ('forward', 'red', None, 'forward'), 'deadline': 20, 't': 5, 'action': None, 'reward': 2.5643614815125133, 'waypoint': 'forward'}
Agent previous state: ('forward', 'red', None, 'forward')
Agent properly idled at a red light. (rewarded 2.56)
76% of time remaining to reach destination.

/-------------------
| Step 6 Results
\-------------------

Environment.step(): t = 6
Environment.act() [POST]: location: (8, 3), heading: (1, 0), action: None, reward: 1.4155477778
Environment.act(): Step data: {'inputs'

Agent properly idled at a red light. (rewarded 1.36)
55% of time remaining to reach destination.

/-------------------
| Step 9 Results
\-------------------

Environment.step(): t = 9
Environment.act() [POST]: location: (5, 3), heading: (0, -1), action: left, reward: 1.86235757542
Environment.act(): Step data: {'inputs': {'light': 'green', 'oncoming': None, 'right': 'left', 'left': None}, 'violation': 0, 'light': 'green', 'state': ('left', 'green', None, None), 'deadline': 11, 't': 9, 'action': 'left', 'reward': 1.8623575754227448, 'waypoint': 'left'}
Agent previous state: ('left', 'green', None, None)
Agent followed the waypoint left. (rewarded 1.86)
50% of time remaining to reach destination.

/-------------------
| Step 10 Results
\-------------------

Environment.step(): t = 10
Environment.act() [POST]: location: (6, 3), heading: (1, 0), action: right, reward: 1.7314082884
Environment.act(): Step data: {'inputs': {'light': 'red', 'oncoming': 'forward', 'right': None, 'left': None},